In [72]:

import random
import re
from dataclasses import dataclass

import numpy as np
import torch as t
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset
import einops
from fancy_einsum import einsum
from torchtyping import TensorType
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

device = t.device("cuda" if t.cuda.is_available() else "cpu")

In [3]:
# From previous exercises

class PositionalEncoding(nn.Module):
    pe: t.Tensor

    def __init__(self, d_model: int, max_len: int = 5000, dropout: float = 0.1):
        self.d_model = d_model
        self.max_len = max_len
        self.dropout = dropout
        
        super().__init__()

        self.register_buffer(
            "pe",    
            self.encode(self.max_len, self.d_model)
        )

    def encode(self, seq_len: int, embedding_dim: int) -> t.Tensor:
        raise NotImplementedError

    def forward(self, x: t.Tensor) -> t.Tensor:
        '''
        x: Tensor, shape [batch, seq_len, embedding_dim]
        '''

        _, seq_len, embedding_dim = x.shape

        return x + self.pe[:seq_len, :embedding_dim].unsqueeze(0)


class SinusoidalPositionalEncoding(PositionalEncoding):
    def encode(self, seq_len: int, embedding_dim: int) -> t.Tensor:
        i = t.arange(seq_len).unsqueeze(1)
        d = t.arange(embedding_dim).unsqueeze(0)

        return (
            t.sin(i / 10000 ** (d / embedding_dim)) * (d % 2 == 0)
            + t.cos(i / 10000 ** ((d - 1) / embedding_dim)) * (d % 2 == 1)
        )

def mask(A: TensorType[..., "seq_len", "seq_len"]) -> TensorType[..., "seq_len", "seq_len"]:
    seq_len = A.shape[-1]

    mask = t.triu(t.ones(seq_len, seq_len), diagonal=1).bool()
    return A.masked_fill(mask, -np.inf)

def multihead_masked_attention(
    Q: TensorType["batch", "seq", "n_heads*headsize"], 
    K: TensorType["batch", "seq", "n_heads*headsize"], 
    V: TensorType["batch", "seq", "n_heads*headsize"],
    num_heads: int
) -> TensorType["batch", "seq", "n_heads*headsize"]:
    '''
    Should return the results of multihead self-attention.

    Q: shape (batch, seq, n_heads*headsize)
    K: shape (batch, seq, n_heads*headsize)
    V: shape (batch, seq, n_heads*headsize)
    num_heads: int

    Return: shape (batch, seq, n_heads*headsize)
    '''
    _Q = einops.rearrange(Q, "b s (n h) -> b n s h", n=num_heads)    
    _K = einops.rearrange(K, "b s (n h) -> b n s h", n=num_heads)    
    _V = einops.rearrange(V, "b s (n h) -> b n s h", n=num_heads)

    d_head = _Q.shape[-1]

    A_pre: TensorType["b", "n", "s_q", "s_k"] = mask(
        einsum("b n s_q h, b n s_k h -> b n s_q s_k", _Q, _K)
    ) / np.sqrt(d_head)

    A: TensorType["b", "n", "s_q", "s_k"] = t.softmax(A_pre, dim=-1)
    AV: TensorType["b", "n", "s_q", "h"] = einsum("b n s_q s_k, b n s_k h -> b n s_q h", A, _V)

    return einops.rearrange(AV, "b n s h -> b s (n h)") 


class MultiheadMaskedAttention(nn.Module):
    W_QKV: nn.Linear
    W_O: nn.Linear

    def __init__(self, hidden_size: int, num_heads: int):
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_size = hidden_size // num_heads

        super().__init__()

        self.W_QKV = nn.Linear(hidden_size, hidden_size * 3)
        self.W_O = nn.Linear(hidden_size, hidden_size)

    def forward(self, x: TensorType["batch", "seq", "hidden_size"]) -> TensorType["batch", "seq", "hidden_size"]:
        '''
        x: shape (batch, seq, hidden_size)

        Return: shape (batch, seq, hidden_size)
        '''
        Q, K, V = self.W_QKV(x).chunk(3, dim=-1)        
        return self.W_O(multihead_masked_attention(Q, K, V, self.num_heads))


@dataclass(frozen=True)
class TransformerConfig:
    '''Constants used throughout your decoder-only transformer model.'''

    num_layers: int = 6
    num_heads: int = 8
    vocab_size: int = 256
    hidden_size: int = 512
    max_seq_len: int = 512
    dropout: float = 0.1
    layer_norm_epsilon: float = 1e-05

config = TransformerConfig()

class MLPBlock(nn.Module):

    def __init__(self, hidden_size: int, dropout: float):
        self.hidden_size = hidden_size

        super().__init__()

        self.linear1 = nn.Linear(hidden_size, hidden_size * 4)
        self.gelu = nn.GELU()
        self.linear2 = nn.Linear(hidden_size * 4, hidden_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: t.Tensor) -> t.Tensor:
        x = self.linear1(x)
        x = self.gelu(x)
        x = self.linear2(x)
        x = self.dropout(x)

        return x

class DecoderBlock(nn.Module):

    def __init__(self, hidden_size: int, num_heads: int, layer_norm_epsilon: float, dropout: float):
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.layer_norm_epsilon = layer_norm_epsilon
        self.dropout = dropout

        super().__init__()

        self.attention = MultiheadMaskedAttention(hidden_size, num_heads)
        self.ln1 = nn.LayerNorm(hidden_size, eps=layer_norm_epsilon)
        self.mlp = MLPBlock(hidden_size, dropout)
        self.ln2 = nn.LayerNorm(hidden_size, eps=layer_norm_epsilon)

    def forward(self, x: t.Tensor) -> t.Tensor:
        x = x + self.attention(self.ln1(x))
        x = x + self.mlp(self.ln2(x))

        return x


class DecoderOnlyTransformer(nn.Module):

    def __init__(self, config: TransformerConfig):
        self.config = config

        super().__init__()

        self.embedding = nn.Embedding(config.vocab_size, config.hidden_size)
        self.positional_embedding = SinusoidalPositionalEncoding(config.hidden_size, config.max_seq_len)

        self.dropout = nn.Dropout(config.dropout)
        self.decoder_blocks = nn.ModuleList([
            DecoderBlock(config.hidden_size, config.num_heads, config.layer_norm_epsilon, config.dropout)
            for _ in range(config.num_layers)
        ])
        self.ln = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_epsilon)
        self.unembed = nn.Linear(config.hidden_size, config.vocab_size)        
        self.softmax = nn.Softmax(dim=2)

    def forward(self, x: t.Tensor) -> t.Tensor:
        x = self.embedding(x)
        x = self.positional_embedding(x)
        x = self.dropout(x)

        for decoder_block in self.decoder_blocks:
            x = decoder_block(x)
        
        x = self.ln(x)
        x = self.unembed(x)
        x = self.softmax(x)

        return x


# %%


In [62]:

from typing import Optional, Union


with open("./shakespeare.txt", "r") as f:
    shakespeare = f.read()

class WordsTokenizer():
    def __init__(self, corpus: str):
        self.from_tokens = list(set(re.split(r"\b", corpus)))
        self.to_tokens = {k: i for i, k in enumerate(self.from_tokens)}
        self.vocab_size = len(self.from_tokens)

    def encode(self, initial_text: str, return_tensors: Optional[str] = None) -> Union[list, np.ndarray, t.Tensor]:
        tensors_list = [self.to_tokens[s] for s in re.split(r"\b", initial_text) if len(s) > 0]
        if return_tensors is None:
            return tensors_list
        elif return_tensors == "pt":
            return t.tensor(tensors_list)
        elif return_tensors == "np":
            return np.array(tensors_list)
        else:
            raise Exception("Unexpected value for `return_tensors`.")

    def decode(self, list_of_ids: Union[t.Tensor, list[int]]) -> str:
        return ''.join([self.from_tokens[token] for token in list_of_ids])

tokenizer = WordsTokenizer(shakespeare)
shakespeare_encoded = tokenizer.encode(shakespeare, return_tensors="pt")
print(tokenizer.from_tokens[:10], len(tokenizer.from_tokens))

['dispossess', 'inscrutable', 'cuisses', 'sonance', 'Offers', 'bequeathing', 'Accompanied', 'Ruffian', 'She', 'Cotshall'] 34543


In [63]:

class ShakespeareDataset(Dataset):
    def __init__(self, tokens: t.Tensor | list[int], seq_len: int):
        self.inputs = []
        self.targets = []

        for i in range(0, len(tokens) - seq_len - 1, seq_len):
            self.inputs.append(tokens[i : i + seq_len])
            self.targets.append(tokens[i + 1 : i + seq_len + 1])

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx: int):
        input_ = self.inputs[idx]
        target = self.targets[idx]
        return input_, target

train_corpus = shakespeare_encoded[:int(len(shakespeare_encoded) * 0.8)]
test_corpus = shakespeare_encoded[int(len(shakespeare_encoded) * 0.8) :]

train_data = ShakespeareDataset(train_corpus, 128) # type: ignore
test_data = ShakespeareDataset(test_corpus, 128) # type: ignore

train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [69]:

config = TransformerConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=128,
    num_layers=4,
    num_heads=4,
    max_seq_len=128
)

transformer = DecoderOnlyTransformer(config)

In [73]:

def train(model, optimizer, loss_fn, trainloader, epochs, dataset_name=None, plot=True):

    loss_list = []

    for epoch in range(epochs):
        
        progress_bar = tqdm(trainloader)
        
        for (x, y) in progress_bar:
            x = x.to(device)
            y = y.to(device)

            optimizer.zero_grad()
            
            logits = model(x)
            # logits dimensions are (batch, seq, digits), but we care about probabilities for each digit
            # so we need to reshape into (batch * seq, digits)
            loss = loss_fn(einops.rearrange(logits, "b s d -> (b s) d"), y.flatten())
            loss.backward()

            optimizer.step()
            
            progress_bar.set_description(f"epoch = {epoch+1}, loss = {loss.item():.4f}")

            loss_list.append(loss.item())
    
        if plot:
            plt.plot(loss_list)
            plt.title(f"{dataset_name} loss")
            plt.show()
    
    return model    

In [74]:
optimizer = t.optim.Adam(transformer.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

train(transformer, optimizer, loss_fn, train_dataloader, 10)

  0%|          | 0/195 [00:00<?, ?it/s]

KeyboardInterrupt: 